In [61]:
from src.exp_helper import *
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV,RidgeCV
import geopandas as gp
from scipy.stats import wilcoxon, pearsonr
from src.utils.metric_single_num import rmse

In [41]:

V = 1

D = CompileData(verbose=V, spu_name='bnia_nbh')
D.set_x(['crime'], by_category=True)
D.set_y('crime/burglary')

#

loading data crime
adding spatial unit assignment to crime data
spatial unit assignment for data crime-train in spu bnia_nbh existed
****WARNING**** Some data get multiple assignments
****WARNING**** Some data get 0 assignment
spatial unit assignment for data crime-dev in spu bnia_nbh existed
****WARNING**** Some data get multiple assignments
****WARNING**** Some data get 0 assignment
set_x: adding individual category of data crime
all categories in data: crime are: {'shooting', 'robbery', 'homicide', 'theft_larceny', 'rape', 'arson', 'burglary', 'assault'}.
set data for y, dname/categories=crime/burglary


In [46]:
import datetime

In [35]:
from collections import defaultdict

In [34]:
indicators = pd.read_excel('data/open-baltimore/raw/VS16_Indicators_2010-2016.xlsx',sheet_name='2015')
indicators = indicators.set_index('CSA2010').drop('Baltimore City')

In [54]:

T_R = Rolling(rsd='2015-01-01', red='2016-07-01', rstep=7, tw_past=60, tw_pred=7)
E_R = Rolling(rsd='2016-07-01', red='2017-07-01', rstep=7, tw_past=60, tw_pred=7)

In [55]:
M = SVR(gamma='scale')
M = LassoCV(cv=5)
M=RidgeCV(cv=5)
X, Y = data_for_fit(D, roller=T_R, x_setting='event_cnt', y_setting='event_cnt', stack_roll=False,
                    verbose=V)

M.fit(X, Y)
print(M)
print('training score: ', M.score(X, Y))

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=5, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)
training score:  0.5434456886073633


In [56]:
dates = E_R.roll()

In [58]:
def get_coi(d,m,x,y,indicators):
    nb = D.spu.copy()
    nb['pred'] = M.predict(x)
    nb['true'] = y
    nb.set_index('CSA2010', inplace=True)
    nb = nb.merge(indicators,on='CSA2010')

    x_cols = indicators.columns.tolist()
    res = []
    for xc in x_cols:
        cond = 'Household' in xc
        if not cond: continue
        idctr = nb[xc]
        rt,pt = pearsonr(nb['true'],idctr)
        rp,pp = pearsonr(nb['pred'],idctr)
        res.append({'xc':xc, 'rt':rt, 'abs_rt': abs(rt), 'pt':pt, 'rp':rp, 'abs_rp': abs(rp), 'pp':pp})
    #         break
    #     break

    res_df = pd.DataFrame(res)[['xc','rt','abs_rt','pt', 'rp', 'abs_rp', 'pp']]

    return res_df[(res_df.pp<0.05) & (res_df.pt>0.05)][['xc','pp','rp']]

In [65]:
y

array([ 8.,  4.,  8.,  7.,  0.,  5.,  3.,  0.,  0.,  1.,  0.,  0.,  6.,
        5.,  2.,  0.,  6.,  2.,  3.,  3.,  2.,  1.,  3.,  0.,  7.,  0.,
        2.,  1.,  2.,  2.,  8.,  4.,  2.,  4.,  0.,  1.,  1.,  0., 12.,
        1.,  2.,  2.,  4.,  0.,  0.,  2., 10.,  5.,  1.,  2.,  3.,  0.,
        0.,  7.,  1.])

In [68]:
res = defaultdict(dict)

for i, d in enumerate(dates):
    rsd = d[0]
    rsd = datetime.datetime.strftime(rsd, '%Y-%m-%d')
    red = d[3] + datetime.timedelta(seconds=1)
    red = datetime.datetime.strftime(red, '%Y-%m-%d')
    period = '%s~%s' % (rsd,red)
    new_roller = Rolling(rsd=rsd, red=red, rstep=7, tw_past=60, tw_pred=7)
    x,y = data_for_fit(D, roller=new_roller, x_setting='event_cnt', y_setting='event_cnt', stack_roll=False, verbose=V)
    score=M.score(x,y)
#     print(period,M.score(x,y))
    coi = get_coi(D,M,x,y,indicators)
    for _, (c,p,r) in get_coi(D,M,x,y,indicators).iterrows():
#         print(c,p)
        res[c][period]='pear=%.3f, p=%.3f, pred_r2=%.3f' %(r,p,score)
    if i>=20:
        break
#     break

In [69]:
res_df = pd.DataFrame.from_dict(res,'index')
res_df['notnull'] = res_df.apply(lambda x: (~x.isnull()).sum(), axis=1)/res_df.shape[1]
res_df.sort_values('notnull', ascending=False)

,2016-07-01~2016-09-06,2016-07-08~2016-09-13,2016-07-15~2016-09-20,2016-07-22~2016-09-27,2016-09-02~2016-11-08,2016-09-30~2016-12-06,2016-11-11~2017-01-17,2016-07-29~2016-10-04,2016-08-05~2016-10-11,2016-08-26~2016-11-01,2016-09-09~2016-11-15,2016-09-16~2016-11-22,2016-09-23~2016-11-29,2016-10-07~2016-12-13,2016-11-18~2017-01-24,2016-11-04~2017-01-10,notnull
Percent of Households with No Vehicles Available,"pear=0.373, p=0.005, pred_r2=0.250","pear=0.365, p=0.006, pred_r2=0.121",NaN,"pear=0.359, p=0.007, pred_r2=0.354","pear=0.334, p=0.013, pred_r2=0.202","pear=0.290, p=0.032, pred_r2=0.038","pear=0.270, p=0.046, pred_r2=0.299","pear=0.322, p=0.017, pred_r2=0.197","pear=0.308, p=0.022, pred_r2=0.316","pear=0.331, p=0.014, pred_r2=-0.016","pear=0.301, p=0.026, pred_r2=0.286","pear=0.273, p=0.044, pred_r2=0.343",NaN,"pear=0.290, p=0.032, pred_r2=0.162","pear=0.285, p=0.035, pred_r2=0.583",NaN,0.8125
Percent of Family Households Living Below the Poverty Line,"pear=0.295, p=0.029, pred_r2=0.250","pear=0.288, p=0.033, pred_r2=0.121",NaN,"pear=0.284, p=0.036, pred_r2=0.354","pear=0.322, p=0.017, pred_r2=0.202","pear=0.302, p=0.025, pred_r2=0.038",NaN,NaN,NaN,"pear=0.308, p=0.022, pred_r2=-0.016","pear=0.308, p=0.022, pred_r2=0.286","pear=0.283, p=0.037, pred_r2=0.343","pear=0.296, p=0.028, pred_r2=0.148","pear=0.296, p=0.028, pred_r2=0.162",NaN,NaN,0.6250
"Percent of Households Earning Less than $25,000","pear=0.327, p=0.015, pred_r2=0.250","pear=0.326, p=0.015, pred_r2=0.121",NaN,"pear=0.324, p=0.016, pred_r2=0.354","pear=0.311, p=0.021, pred_r2=0.202","pear=0.274, p=0.043, pred_r2=0.038",NaN,"pear=0.284, p=0.036, pred_r2=0.197","pear=0.274, p=0.043, pred_r2=0.316","pear=0.310, p=0.021, pred_r2=-0.016","pear=0.274, p=0.043, pred_r2=0.286",NaN,NaN,NaN,NaN,NaN,0.5625
Median Household Income,"pear=-0.318, p=0.018, pred_r2=0.250","pear=-0.313, p=0.020, pred_r2=0.121","pear=-0.314, p=0.019, pred_r2=0.097","pear=-0.301, p=0.026, pred_r2=0.354","pear=-0.271, p=0.045, pred_r2=0.202","pear=-0.282, p=0.037, pred_r2=0.038","pear=-0.280, p=0.039, pred_r2=0.299",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4375
Average Household Size,NaN,NaN,NaN,NaN,NaN,"pear=0.295, p=0.029, pred_r2=0.038","pear=0.307, p=0.022, pred_r2=0.299",NaN,NaN,NaN,NaN,NaN,NaN,"pear=0.281, p=0.037, pred_r2=0.162","pear=0.313, p=0.020, pred_r2=0.583","pear=0.315, p=0.019, pred_r2=0.370",0.3125
"Percent of Households Earning More than $75,000","pear=-0.309, p=0.022, pred_r2=0.250","pear=-0.299, p=0.026, pred_r2=0.121","pear=-0.297, p=0.028, pred_r2=0.097","pear=-0.284, p=0.036, pred_r2=0.354",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2500
Total Number of Households,NaN,NaN,NaN,NaN,NaN,NaN,"pear=0.492, p=0.000, pred_r2=0.299",NaN,NaN,NaN,NaN,NaN,"pear=0.406, p=0.002, pred_r2=0.148",NaN,NaN,NaN,0.1250
Percent of Households with Children Under 18,NaN,NaN,NaN,NaN,NaN,"pear=0.271, p=0.046, pred_r2=0.038",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0625
